# 15 min Intro to Persistent Homology
### <a href="https://ben300694.github.io/" target="_blank">Benjamin Matthias Ruppik</a>, Max-Planck Institute for Mathematics, Bonn

## Filtrations of topological spaces and homology groups

A lot of topological spaces $X$ come with a natural filtration,
that is, an exhaustion by subsets $U_{i}$ with the property
that for $i < j$ we have $U_{i} \subseteq U_{j}$

<img src="img/surface_morse_position.jpg" width=400 />

## Importing and preprocessing the data

In [2]:
[1,3,5,6]

[1, 3, 5, 6]

**TODO**

## Applying persistent homology to the data

persist = to stick around for a long time

homology = cycles module boundaries

persistent homology measures non-trivial cycles which can be detected in significant parts ofthe filtrations

## Conclusion

### Properties of the data we could detect with persistent homology

 * Large scale features
 * **TODO**

### Drawbacks of using persistent homology
 * Low resolution
 * **TODO**